In [9]:
import os 
import glob
import rioxarray as rxr 
import xarray as xr
import numpy as np
forcings_path = os.environ['STORE'] + '/forcings'
scratch = os.environ['SCRATCH']

In [20]:
aerosol_inca = xr.open_mfdataset(f"{forcings_path}/aerosols*_from_inca.nc",engine='netcdf4')
surface_fluxes = xr.open_mfdataset(f"{forcings_path}/Ndep_input4MIPs_surfaceFluxes_CMIP_NCAR-CCMI-2-0_gn_*-clim.nc",decode_times=False)
inputs = xr.open_dataset(f'{scratch}/1960_1.nc')

inputs.rio.write_crs('4326',inplace=True)
surface_fluxes.rio.write_crs('4326',inplace=True)
surface_fluxes = surface_fluxes.drop('time_bnds')
surface_fluxes = surface_fluxes.rename({'lat':'y','lon':'x'})
inputs = inputs.rename({'lat':'y','lon':'x'})
surface_fluxes.rio.set_spatial_dims(x_dim='x',y_dim='y',inplace=True)
inputs.rio.set_spatial_dims(x_dim='x',y_dim='y',inplace=True)

surface_fluxes.rio.reproject_match(inputs)
#surface_fluxes['ndep'].isel(time=0).plot()

/tmp/ipykernel_2014839/926716338.py:7: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  surface_fluxes = surface_fluxes.drop('time_bnds')


<xarray.Dataset> Size: 119MB
Dimensions:      (time: 288, x: 144, y: 143)
Coordinates:
  * time         (time) float64 2kB 1.332e+03 1.334e+03 ... 1.618e+03 1.62e+03
    spatial_ref  int64 8B 0
  * x            (x) float32 576B 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * y            (y) float32 572B -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
Data variables:
    wetnhx       (time, y, x) float32 24MB 1.505e-17 1.505e-17 ... 3.936e-14
    wetnoy       (time, y, x) float32 24MB 1.524e-15 1.524e-15 ... 1.129e-13
    drynhx       (time, y, x) float32 24MB 1.529e-16 1.487e-16 ... 3.373e-15
    drynoy       (time, y, x) float32 24MB 5.036e-17 5.034e-17 ... 4.024e-21
    ndep         (time, y, x) float32 24MB 1.742e-15 1.738e-15 ... 1.556e-13
Attributes: (12/24)
    variable_id:               drynoy
    Conventions:               CF-1.6
    activity_id:               input4MIPs
    contact:                   Douglas Kinnison (dkin@ucar.edu); Michaela Heg...
    creation_date:             2016-11-15T12:00:00Z
    dataset_category:          surfaceFluxes
    ...                        ...
    references:                Hegglin, M. I., D. Kinnison, J.-F. Lamarque, e...
    license:                   CCMI data produced by NCAR is licensed under a...
    history:                   Fri Oct 20 13:44:11 2017: /usr/bin/ncatted -O ...
    comment:                   Data product from CAM-Chem.
    NCO:                       4.3.7
    nco_openmp_thread_number:  1

In [10]:
import pandas as pd 

atmos_forcings = [f'{forcings_path}/CH4_CMIP6_0000_2014.txt',
                  f'{forcings_path}/CFC11eq_CMIP6_0000_2014.txt',
                  f'{forcings_path}/CFC12eq_CMIP6_0000_2014.txt',
                  f'{forcings_path}/CO2_CMIP6_0000_2014.txt']
# methane_forcings = pd.read_csv(f'{forcings_path}/CH4_CMIP6_0000_2014.txt',delimiter='\t')
# cfc11_forcings = pd.read_csv(f'{forcings_path}/CFC11eq_CMIP6_0000_2014.txt',delimiter='\t')
# cfc12_forcings = pd.read_csv(f'{forcings_path}/CFC12eq_CMIP6_0000_2014.txt',delimiter='\t')
# co2_forcings = pd.read_csv(f'{forcings_path}/CO2_CMIP6_0000_2014.txt',delimiter='\t')

# methane_forcings.columns=['year','data']


In [15]:
agg_atmos_forcings = []
for file in atmos_forcings:
    forcings = pd.read_csv(file,delimiter='\t')
    forcings.columns=['year','data']
    start_year = 1960
    end_year = 2015
    blank=np.ndarray((end_year - start_year))
    for i in range(start_year,end_year):
        #need i-1 in selector because indexing is off by one in the pandas
        blank[i-start_year]= forcings.iloc[i-1]['data']
    agg_atmos_forcings.append(blank)

In [18]:
stacked = np.stack(agg_atmos_forcings)
#np.save('atmos_forcings',np.stack(agg_atmos_forcings))

In [26]:
np.broadcast_to(np.expand_dims(stacked[:,0],(1,2,3)),(4,12,144,143)).shape

(4, 12, 144, 143)

In [37]:
blank=np.ndarray((12,144,143))
blank[:,:,:]= methane_forcings.iloc[2013]['data']

In [13]:
solar = xr.open_dataset(f'{forcings_path}/solarforcing_v3.1_daily_6bands_1984.nc')
ssi = np.broadcast_to(np.expand_dims(solar.groupby('time.month').mean()['ssi'].data,(-1,-2)),(12,6,144,143))
pft_map = xr.open_dataset(f'{forcings_path}/PFTmap_1984.nc',decode_times=False)
pft_map = pft_map.rio.write_crs('4326')
pft_map = pft_map.drop_dims('time_counter')
pft_map = pft_map.rename({'lon':'y','lat':'x'})
pft_map = pft_map.rio.set_spatial_dims(x_dim='x',y_dim='y')
inputs = inputs.rio.set_spatial_dims(x_dim='x',y_dim='y')
pft_map.rio.reproject_match(inputs)

/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/xarray/coding/times.py:170: SerializationWarning: Ambiguous reference date string: 0-1-1. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0000-1-1). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


MissingSpatialDimensionError: x dimension (x) not found.

In [1]:
import numpy as np

In [3]:
atmos_forcings_loaded = np.load('../data/atmos_forcings.npy')

In [4]:
atmos_forcings_loaded

array([[1264.1  , 1269.5  , 1282.6  , 1300.8  , 1317.4  , 1331.1  ,
        1342.2  , 1354.3  , 1371.7  , 1389.3  , 1411.1  , 1431.1  ,
        1449.3  , 1462.9  , 1476.1  , 1491.7  , 1509.1  , 1527.7  ,
        1546.9  , 1566.2  , 1584.9  , 1602.7  , 1618.7  , 1632.6  ,
        1643.5  , 1655.9  , 1668.8  , 1683.7  , 1693.9  , 1705.6  ,
        1717.4  , 1729.3  , 1740.1  , 1743.1  , 1748.6  , 1755.2  ,
        1757.2  , 1761.5  , 1770.3  , 1778.2  , 1778.   , 1776.5  ,
        1779.   , 1783.6  , 1784.2  , 1783.4  , 1783.4  , 1789.   ,
        1798.4  , 1802.1  , 1807.9  , 1813.1  , 1815.3  , 1822.6  ,
        1831.5  ],
       [  14.875,   15.064,   15.272,   15.506,   15.753,   16.012,
          16.288,   16.588,   16.938,   17.314,   17.721,   18.174,
          18.679,   19.229,   19.837,   20.519,   21.18 ,   21.906,
          22.728,   23.815,   24.953,   26.114,   27.229,   28.249,
          29.275,   30.423,   31.606,   32.783,   34.1  ,   35.467,
          36.865,   38.365,  